# DEFORESTATION SCENARIO PIPELINE

In [1]:
import digitalhub as dh
PROJECT_NAME = "deforestation"
proj = dh.get_or_create_project(PROJECT_NAME) # source="git://github.com/scc-digitalhu

Register to the open data space copenicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [ ]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

### Download data from Sentinel 2

In [3]:
string_dict_data = """{
 "satelliteParams":{
     "satelliteType": "Sentinel2"
 },
 "startDate": "2018-01-01",
 "endDate": "2019-12-31",
 "geometry": "POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))",
 "area_sampling": "true",
 "cloudCover": "[0,20]",
 "artifact_name": "data" 
 }"""

list_args =  ["main.py",string_dict_data]
function_s2 = proj.new_function("download_images_s2",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.10.1",command="python")

In [ ]:
run = function_s2.run(action="job",
        secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
        fs_group='8877',
        args=list_args,           
        volumes=[{
            "volume_type": "persistent_volume_claim",
            "name": "volume-deforestation",
            "mount_path": "/app/files",
            "spec": {
                "size": "300Gi"
            }}])

In [29]:
run.refresh().status.state

'RUNNING'

In [11]:
artifact = proj.get_artifact('data')

### Perform Data Elaboration on RS-Tool

#### Input data (Shape file)

In [ ]:
artifact_name='bosco'
src_path='bosco'
artifact_bosco = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

#### Input data (Tiles)

In [4]:
artifact_name='data'
src_path='data'
artifact_data = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

In [2]:
function_rs = proj.new_function("elaborate",kind="container", image="ghcr.io/tn-aixpa/rs-deforestation:2.7_b2", code_src="launch.sh")

In [ ]:
#!pip uninstall digitalhub -y && pip install git+https://github.com/scc-digitalhub/digitalhub-sdk
!pip uninstall digitalhub digitalhub-runtime-container -y && pip install git+https://github.com/scc-digitalhub/digitalhub-sdk git+https://github.com/scc-digitalhub/digitalhub-sdk-runtime-container 

In [ ]:
run_el = function_rs.run(action="job",
            fs_group='8877',
            resources={
                "cpu": {
                    "requests": "3",
                    
                },
                "mem":{
                    "requests": "16Gi"
                }
            },
            profile="1xa100",
            volumes=[{
            "volume_type": "persistent_volume_claim",            
            "name": "volume-deforestation", # this name has to be equal to the name of the volume created in krm
            "mount_path": "/app/files",
            "spec": {
                "size": "5Gi"
            }}],
            args=['/shared/launch.sh', 'bosco','data',"['2018','2019']",'deforestation_output']
         )

In [8]:
run_el.refresh().status.state

'RUNNING'

In [11]:
startYear=2018
endYear=2019
geometry="POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))"
string_dict_data = """{"satelliteParams":{"satelliteType": "Sentinel2" }, "startDate": \"""" + str(startYear) + """-01-01\", "endDate": \"""" + str(endYear) + """-12-31\", "geometry": \"""" + str(geometry) + """\", "area_sampling": "true", "cloudCover": "[0,20]", "artifact_name": "data_s2_deforestation"}"""
string_dict_data

'{"satelliteParams":{"satelliteType": "Sentinel2" }, "startDate": "2018-01-01", "endDate": "2019-12-31", "geometry": "POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))", "area_sampling": "true", "cloudCover": "[0,20]", "artifact_name": "data_s2_deforestation"}'

In [14]:
yearList="[" + str(startYear) + ',' + str(endYear) + "]"
yearList

'[2018,2019]'

In [24]:
%%writefile "deforestation_pipeline.py"


from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(startYear, endYear, geometry, input1, input2, outputName):
    with pipeline_context() as pc:
        string_dict_data = """{"satelliteParams":{"satelliteType": "Sentinel2" }, "startDate": \"""" + str(startYear) + """-01-01\", "endDate": \"""" + str(endYear) + """-12-31\", "geometry": \"""" + str(geometry) + """\", "area_sampling": "true", "cloudCover": "[0,20]", "artifact_name": "data_s2_deforestation"}"""
        list_args =  ["main.py",string_dict_data]
        s1 = pc.step(name="download",function="download_images_s2", action="job", secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"], fs_group='8877', args=list_args, profile="1xa100", volumes=[{"volume_type": "persistent_volume_claim","name": "volume-deforestation", "mount_path": "/app/files","spec": {"size": "350Gi"}}])
        s2 = pc.step(name="elaborate", function="elaborate", action="job", fs_group='8877', resources={"cpu": {"requests": "3"},"mem":{"requests": "16Gi"}},profile="1xa100",volumes=[{"volume_type": "persistent_volume_claim","name": "volume-deforestation", "mount_path": "/app/files","spec": {"size": "5Gi"}}],args=['/shared/launch.sh', input1, 'data_s2_deforestation',"[" + str(startYear) + ',' + str(endYear) + "]",outputName]).after(s1)        
     


Overwriting deforestation_pipeline.py


In [25]:
workflow = proj.new_workflow(name="pipeline_deforestation", kind="kfp", code_src= "deforestation_pipeline.py", handler = "myhandler")

In [26]:
workflow.run(action="build", wait=True)

2025-05-22 07:32:54,250 - INFO - Waiting for run 58cb5cd8e3774a5e9f7e9c75e8955c1b to finish...
2025-05-22 07:32:59,286 - INFO - Waiting for run 58cb5cd8e3774a5e9f7e9c75e8955c1b to finish...
2025-05-22 07:33:04,309 - INFO - Waiting for run 58cb5cd8e3774a5e9f7e9c75e8955c1b to finish...
2025-05-22 07:33:09,356 - INFO - Run 58cb5cd8e3774a5e9f7e9c75e8955c1b finished in 15.11 seconds.


{'kind': 'kfp+run', 'metadata': {'project': 'deforestation', 'name': '58cb5cd8e3774a5e9f7e9c75e8955c1b', 'created': '2025-05-22T07:32:54.228Z', 'updated': '2025-05-22T07:32:59.875Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu', 'relationships': [{'type': 'run_of', 'dest': 'store://deforestation/workflow/kfp/pipeline_deforestation:e63c9cb9668f49b8a09c5bbc94db36e1'}]}, 'spec': {'task': 'kfp+build://deforestation/364c7c84bc9b4b498b3de05ac7bf24b0', 'local_execution': False, 'workflow': 'kfp://deforestation/pipeline_deforestation:e63c9cb9668f49b8a09c5bbc94db36e1', 'source': {'source': 'deforestation_pipeline.py', 'handler': 'myhandler', 'base64': 'Cgpmcm9tIGRpZ2l0YWxodWJfcnVudGltZV9rZnAuZHNsIGltcG9ydCBwaXBlbGluZV9jb250ZXh0CgpkZWYgbXloYW5kbGVyKHN0YXJ0WWVhciwgZW5kWWVhciwgZ2VvbWV0cnksIGlucHV0MSwgaW5wdXQyLCBvdXRwdXROYW1lKToKICAgIHdpdGggcGlwZWxpbmVfY29udGV4dCgpIGFzIHBjOgogICAgICAgIHN0cmluZ19kaWN0X2RhdGEgPSAiIiJ7InNhdGVsbGl0ZVBhcmFtcyI6eyJzYXRlbGxpdGVUeXBlIjogIlNlbnRpbmVsMiIg

In [ ]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startYear": 2018,
    "endYear": 2019,
    "geometry": "POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))",
    "input1":"bosco",
    'input2': "data_s2_deforestation",
    "outputName": "deforestation_2018_19"
    },
    wait=True)

2025-05-22 07:33:28,775 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:33:33,794 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:33:38,832 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:33:43,864 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:33:48,893 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:33:53,916 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:33:58,946 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:34:03,977 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:34:09,005 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:34:14,034 - INFO - Waiting for run bb594b2bf6e14bdeb3406e652208991e to finish...
2025-05-22 07:34:19,058 - INFO - Waiting for run b